## 에이전트를 MCP 도구로 노출하기

이 과정을 통해서 모델 컨텍스트 프로토콜(MCP)을 통해 에이전트를 도구로 노출하여 MCP 도구를 지원하는 다른 시스템에서 사용할 수 있도록 하는 방법을 보여줍니다.

#### 에이전트를 MCP 서버로 노출

as_mcp_server()를 사용하면 에이전트를 MCP 서버로 노출할 수 있습니다. 그러면, MCP 호환 클라이언트에서 에이전트를 도구로 호출할 수 있습니다.
먼저 MCP 서버로 노출할 에이전트를 생성합니다. 물론, 에이전트에 Function 도구를 추가할 수도 있습니다.

In [1]:
from typing import Annotated
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

# 오늘의 스페셜 메뉴를 반환하는 도구
def get_specials() -> Annotated[str, "Returns the specials from the menu."]:
    return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

# 메뉴 아이템의 가격을 반환하는 도구. 9.99 달러로 고정
def get_item_price(
    menu_item: Annotated[str, "The name of the menu item."],
) -> Annotated[str, "Returns the price of the menu item."]:
    return "$9.99"

# RestaurantAgent 에이전트 생성 및 도구 제공
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="RestaurantAgent",
    instructions="Answer questions about the menu.",
    tools=[get_specials, get_item_price],
)

에이전트를 MCP 서버로 전환합니다. 에이전트 이름과 설명은 MCP 서버 메타데이터로 사용됩니다.

In [2]:
# Expose the agent as an MCP server
server = agent.as_mcp_server()

하기의 코드를 실행하여 MCP 서버를 시작합니다. 이제 MCP 호환 클라이언트에서 이 에이전트를 도구로 호출할 수 있습니다.

```python
# 에이전트를 MCP 서버로 전환
server = agent.as_mcp_server()

async def run():
    async with stdio_server() as (read_stream, write_stream):
        await server.run(read_stream, write_stream, server.create_initialization_options())

if __name__ == "__main__":
    anyio.run(run)
```

전체 코드를 노트북에서 실행하기에는 제약이 있어서, 지금까지 다룬 내용을 별도의 Py 파일로 저장하여 실행할 수 있습니다. 완성된 Py 파일은 현재 폴더에서 restaurant_mcp_server.py라는 이름으로 제공됩니다.

### MCP 서버 테스트 (선택사항)

저장된 MCP 서버를 테스트하려면 별도의 터미널에서 실행하세요:

```bash
python MAF/restaurant_mcp_server.py
```

MCP 클라이언트(예: Claude Desktop)에서 이 서버에 연결하여 RestaurantAgent를 도구로 사용할 수 있습니다. 이와 관련된 자세한 테스트 방법은 다음 링크를 참고하시기 바랍니다.

https://github.com/warnov/ms-agent-framework-step-by-step-workshop/blob/main/07-agent-as-MCP-tool/README.md#:~:text=Trying%20the%20server%20from%20Claude%20Desktop